In [17]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from clean_data import clean_data, prepare_data
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [2]:
# Read files
train = pd.read_csv('../cars-competition/data/cars_train.csv', index_col='Id')
submission = pd.read_csv('../cars-competition/data/cars_test.csv', index_col='Id')

In [3]:
# Clean Data
train = clean_data(train)
submission = clean_data(submission)

Cleaning cylinders...
Cleaning condition...
Cleaning odometer...
Cleaning drive...
Cleaning size...
Cleaning manufacturer...
Cleaning fuel...
Cleaning transmission...
Cleaning title status...
Data cleaning complete!
Cleaning cylinders...
Cleaning condition...
Cleaning odometer...
Cleaning drive...
Cleaning size...
Cleaning manufacturer...
Cleaning fuel...
Cleaning transmission...
Cleaning title status...
Data cleaning complete!


In [4]:
# Prepare Data
# columns = ['year','manufacturer','condition','cylinders','fuel','odometer','title_status','transmission','drive','size']
columns = ['year','manufacturer','condition','cylinders','odometer','title_status','transmission','size']
X, y = prepare_data(train,columns,typ='train')
X_sub, _ = prepare_data(submission,columns,typ='test')

In [27]:
# Feature reduction PCA
# n_components = [5,10,15,20,25,30,35,40]
n_components = [5,10,15,20]
for n in n_components:
    exec('pca{0} = PCA(n_components = {0})'.format(n))
    exec('X_new{0} = pca{0}.fit_transform(X)'.format(n))

In [28]:
# Split Train and Test

for n in n_components:
    exec('X_train{0}, X_test{0}, y_train{0}, y_test{0} = train_test_split(X_new{0},y,test_size=0.2,random_state=1)'.format(n))

In [29]:
# DecisionTreeRegressor
for n in n_components:
    exec("dectree_reg{0} = DecisionTreeRegressor()".format(n))
    exec('dectree_reg{0}.fit(X_train{0},y_train{0})'.format(n))

In [30]:
# Predict
for n in n_components:
    exec('y_pred{0} = dectree_reg{0}.predict(X_test{0})'.format(n))

In [31]:
# Check error
error = {}
for n in n_components:
    exec('error[{0}] = mean_squared_error(y_test{0}, y_pred{0})'.format(n))

In [32]:
# Check for optimal feature number for PCA: 25
error

{5: 172109854632549.6,
 10: 185226114512712.44,
 15: 163953821883707.53,
 20: 176987288898988.34}

In [33]:
# Try optimal DecisionTreeRegressor for submission
# PCA
X_sub20 = pca20.transform(X_sub)
# Predict
y_sub20 = dectree_reg20.predict(X_sub20)
y_sub20

array([ 7933.33333333,  2400.        ,  7870.        , ...,
       20000.        ,  1912.25      ,  1900.        ])

In [34]:
# Getting absolute values
y_sub = abs(y_sub35)
y_sub35

NameError: name 'y_sub35' is not defined

In [35]:
# Prepare for submission
sub20 = submission
sub20['price'] = y_sub20
sub20 = sub20['price'] 
sub20.to_csv('../cars-competition/data/sub20_dectree.csv', header=True, index=True)